<a href="https://colab.research.google.com/github/JaquesZanon/JOB/blob/main/Previs%C3%A3o_Demanda_21_10_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install utils
!pip install pmdarima # para autoARIMA
!pip install pyEDM # Para Empirical Dynamic Modeling
!pip install croston # Para modelo croston
!pip install prophet # Para modelo do Facebook prophet
!pip install tqdm
!pip install darts
!pip install 'u8darts[all]'
!pip install skedm

### Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importando os dados

In [ ]:
# Importa aquivo de texto (.txt)
import pandas as pd
#df_original = pd.read_csv('/content/drive/MyDrive/JOB-Martins/DadosPrevisaoDemanda.csv',sep=';', decimal=',', engine='python')
df_original = pd.read_csv('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/PrevisaoDemandaFull.csv',sep=';')

df_original['PrimeiraDataSemana'] = pd.to_datetime(df_original['PrimeiraDataSemana'])
df_original['UltimaDataSemana'] = pd.to_datetime(df_original['UltimaDataSemana'])
df_original = df_original.sort_values(by=['PrimeiraDataSemana'])
df_original.head()

,SiglaUF,DsCategoria,DsSubCategoria,CdGrupoProdutoSimilar,DsGrupoProdutoSimilar,DsMarca,DsDivisaoFornecedor,ano,PrimeiraDataSemana,UltimaDataSemana,SemanaAno,QtdeVendida,VlTotalBruto,VlTotalLiquido,VlReceitaLiquida
350869,BA,ISQUEIRO/FOSFORO/ACENDEDOR,ISQUEIRO,1001857.0,ISQ.BIC MAXI 12X1,Bic,BIC BRASIL S/A,2020.0,2020-07-26,2020-08-01,31.0,1.0,33.61,31.73,25.76
348816,BA,VIDRO,PRATO DE VIDRO,1000659.0,PRATO DX.ASTRAL FD.0800 24X1,Marinex,NADIR FIGUEIREDO IND E COM S/A,2020.0,2020-07-26,2020-08-01,31.0,1.0,78.46,74.69,60.61
348865,BA,SANDALIA,SLIM,1236.0,SAND.HAV.SLIM ORGANIC,Havaianas,ALPARGATAS S/A,2020.0,2020-07-26,2020-08-01,31.0,6.0,121.08,117.42,87.39
348866,BA,INSETICIDA/REPELENTE,INSETICIDA AEROSOL,697.0,INSET.AER. MULTI SBP,SBP,RECKITT BENCKISER - INSETICIDAS,2020.0,2020-07-26,2020-08-01,31.0,1.0,37.41,34.98,28.39
348867,BA,INSETICIDA/REPELENTE,INSETICIDA AEROSOL,697.0,INSET.AER. MULTI SBP,SBP,RECKITT BENCKISER - INSETICIDAS,2020.0,2020-07-26,2020-08-01,31.0,8.0,292.00,268.64,215.70


### Verificar distribuidores e pedidos

In [ ]:
dat_aux = pd.DataFrame(df_original.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux

,CdGrupoProdutoSimilar
DsDivisaoFornecedor,
3M DO BRASIL LTDA,"[819.0, 822.0, 803.0, 818.0, 87.0, 97147.0, 97..."
3M DO BRASIL LTDA - DIV. FITAS,"[408214.0, 400433.0, 400434.0, 400435.0, 40667..."
3M DO BRASIL LTDA - EPI,"[9000524.0, 9000527.0, 9000531.0]"
3M DO BRASIL LTDA - LIMPEZA,"[1702339.0, 1704060.0, 201808.0, 200755.0, 170..."
3M DO BRASIL LTDA LIMPEZA &&MP,"[1702629.0, 1701347.0, 1701261.0, 1701262.0, 1..."
...,...
WHIRLPOOL S.A - L.BCA,"[1062.0, 636.0, 2213230.0, 1809.0, 191.0, 2217..."
WORLD COM IND MED VET COSM LIM,"[701869.0, 701762.0, 701763.0, 701761.0, 70176..."
WOW INDUSTRIA E COMERCIO LTDA,"[1600609.0, 1600715.0, 1600716.0, 4301475.0, 1..."


### Agora importar as bibliotecas

In [ ]:
from scipy import stats
import numpy as np
import pmdarima as pm
#from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from croston import croston
from prophet import Prophet
from tqdm import tqdm
import time
import pyEDM
from darts import TimeSeries
from darts.metrics import mape
from darts.models import (
    NaiveSeasonal,
    NaiveDrift,
    ExponentialSmoothing,
    ARIMA,
    AutoARIMA,
    RegressionEnsembleModel,
    RegressionModel,
    Theta,
    FFT
)
import skedm as edm

In [ ]:
def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

### Se na série temporal exiatir algum feriado, podemos adicioná-lo no modelo prophet.

In [ ]:
# Feriado de independencia
independencia = pd.DataFrame({
  'holiday': 'independencia',
  'ds': pd.to_datetime(['2020-09-13']),
  'lower_window': 0,
  'upper_window': 1,
})
independencia

,holiday,ds,lower_window,upper_window
0,independencia,2020-09-13,0,1


### Extraindo as Séries temporais:
### Com outliers

In [ ]:
df_2 = pd.DataFrame()

total_fornecedor = df_original.DsDivisaoFornecedor.nunique()
num_de_semanas = df_original.SemanaAno.nunique()

for j in range(0, total_fornecedor):
   filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[j]
   df_0 = df_original[filter1]
   df = df_0.groupby(['SemanaAno'])[['VlTotalLiquido']].sum()
   we = pd.DataFrame(list(range(1,num_de_semanas,1)))
   we.columns=['SemanaAno']
   we.index=we.SemanaAno
   df = pd.concat([df, we], axis=1)
   df.columns=[dat_aux.index[j],'SemanaAno']
   #df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=num_de_semanas+1), columns=['date']).iloc[:,0]
   df = pd.DataFrame(df).fillna(0)
   mediana=df.median()

   df = df.drop(['SemanaAno'],axis=1)
   
   #if [mediana==0]:
   #  df.columns = [dat_aux.index[j]]
   #  #df_2[dat_aux.index[j]]=df
   #  df_2 = pd.concat([df_2,df],axis=1)
   #else:
   # Q1 = df.quantile(0.25)
   #  Q3 = df.quantile(0.75)
   ##  IQR = Q3 - Q1
   #  df[((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))] = mediana
   #  df.columns = [dat_aux.index[j]]
    # df_2 = pd.concat([df_2,df],axis=1)
   df_2 = pd.concat([df_2,df],axis=1)


### Sem Outliers

In [ ]:
df_2_sem_out = pd.DataFrame()

total_fornecedor = df_original.DsDivisaoFornecedor.nunique()
num_de_semanas = df_original.SemanaAno.nunique()

for j in range(0, total_fornecedor):
   filter1 = df_original["DsDivisaoFornecedor"]==dat_aux.index[j]
   df_0 = df_original[filter1]
   df = df_0.groupby(['SemanaAno'])[['VlTotalLiquido']].sum()
   we = pd.DataFrame(list(range(1,num_de_semanas,1)))
   we.columns=['SemanaAno']
   we.index=we.SemanaAno
   df = pd.concat([df, we], axis=1)
   df.columns=[dat_aux.index[j],'SemanaAno']
   #df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=num_de_semanas+1), columns=['date']).iloc[:,0]
   df = pd.DataFrame(df).fillna(0)
   mediana=df.median()[0]

   df = df.drop(['SemanaAno'],axis=1)
   
   Q1 = df.quantile(0.25)
   Q3 = df.quantile(0.75)
   IQR = Q3 - Q1
   df[((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))] = mediana
   df_2_sem_out = pd.concat([df_2_sem_out,df],axis=1)


### Calculo ADI & CV2 e categorização das demandas

In [ ]:
ADI={}
CV2={}


for i in range(0, df_2.shape[1]):
  
  ADI[df_2.columns[i]] = 53/sum(df_2.iloc[:,i][df_2.iloc[:,i]!=0])
  CV2[df_2.columns[i]] = (df_2.iloc[:,i].describe()[2]/df_2.iloc[:,i].mean())**2

segmentos = pd.concat([pd.DataFrame.from_dict(ADI,orient='index', columns=['ADI']),
                       pd.DataFrame.from_dict(CV2,orient='index', columns=['CV2'])],axis=1)
cat = np.where([(segmentos.CV2 <= 0.74) & (segmentos.ADI<=2)], 'smooth', 'lumpy')
cat = np.where([(segmentos.CV2 > 0.74) & (segmentos.ADI>2)], 'lumpy', cat)
cat = np.where([(segmentos.CV2 <=0.74) & (segmentos.ADI>2)], 'intermitten', cat)
cat = np.where([(segmentos.CV2 >0.74) & (segmentos.ADI<=2)], 'erratic', cat)
cat = np.transpose(pd.DataFrame(cat))
cat.index = segmentos.index
cat.columns=['cat']
cat_croston = cat[(cat.cat=='lumpy')|(cat.cat=='intermitten')]
cat_rest = cat[(cat.cat!='lumpy') & (cat.cat!='intermitten')]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in double_scalars



### Escolhendo distribuidores apenas 'smooth' e 'erraticos'

In [ ]:
filter = df_original["DsDivisaoFornecedor"].isin(list(cat_rest.index))

df_original1 = df_original[filter]

dat_aux1 = pd.DataFrame(df_original1.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux1

,CdGrupoProdutoSimilar
DsDivisaoFornecedor,
3M DO BRASIL LTDA,"[819.0, 822.0, 803.0, 818.0, 87.0, 97147.0, 97..."
3M DO BRASIL LTDA - DIV. FITAS,"[408214.0, 400433.0, 400434.0, 400435.0, 40667..."
3M DO BRASIL LTDA - LIMPEZA,"[1702339.0, 1704060.0, 201808.0, 200755.0, 170..."
3M DO BRASIL LTDA LIMPEZA &&MP,"[1702629.0, 1701347.0, 1701261.0, 1701262.0, 1..."
3M DO BRASIL LTDA-ESCOLAR,"[801934.0, 801936.0, 800885.0, 800886.0, 80193..."
...,...
WE DO COM.PROD.ELETRONICOS EIRELI,"[3303237.0, 2309845.0, 2309443.0, 2309994.0, 3..."
WHIRLPOOL S.A - L.BCA,"[1062.0, 636.0, 2213230.0, 1809.0, 191.0, 2217..."
WORLD COM IND MED VET COSM LIM,"[701869.0, 701762.0, 701763.0, 701761.0, 70176..."


In [ ]:
#

### Agora o algoritimo para ajustar os modelos. (=~ 1 hora para 293 fornecedores)

In [ ]:
par={}
mape_arima={}
par_croston={}
mape_croston={}
mape_prophet={}
mape_EDM = {}
mape_naive = {}
mape_Theta = {}
mape_FFT = {}

smape_arima={}
smape_croston={}
smape_prophet={}
smape_EDM = {}
smape_naive = {}
smape_Theta = {}
smape_FFT = {}

pred_arima ={}
pred_croston = {}
pred_prophet = {}
pred_EDM = {}
pred_Naive ={}
pred_Theta = {}
pred_FFT = {}

corte_serie=1


for j in tqdm (range(0, df_original1.DsDivisaoFornecedor.nunique()),desc="Processando Fornecedor", ascii=False, ncols=75):
#for j in tqdm (range(0, 2),desc="Processando Fornecedor", ascii=False, ncols=75): 
    try:
         
   # Selecionando os dados para ajustar os modelos
         filter1 = df_original1["DsDivisaoFornecedor"]==dat_aux1.index[j]
         #filter1 = df_original["DsDivisaoFornecedor"]=='NESTLE BRASIL - GAROTO'
         df_0 = df_original1[filter1]
         df = df_0.groupby(['SemanaAno'])[['VlTotalBruto']].sum()
         we = pd.DataFrame(list(range(1,num_de_semanas,1)))
         we.columns=['Semanas']
         we.index=we.Semanas
         df = pd.concat([df, we], axis=1)
         df=df['VlTotalBruto']
         df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=num_de_semanas), columns=['date']).iloc[:,0]
         df = pd.DataFrame(df).fillna(0)
         

      # Substituindo outliers
         mediana=df.VlTotalBruto.median()
         Q1 = df.quantile(0.25)
         Q3 = df.quantile(0.75)
         IQR = Q3 - Q1
         df[((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))] = mediana
          


      # Sets de treino e teste
         #train_len = int(df.shape[0] * corte_serie)
         #train_len = int(corte_serie)
         #train_data, test_data = df[:train_len], df[train_len:]
         test_data = train_data = df

      # Ajustando AutoArima
         stepwise_fit = pm.auto_arima(train_data, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True, d=1, D=1, trace=False,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise
   
      # Armazenando resultados AutoArima
         sentence = [str(dat_aux1.index[j])]
         sentence = '-'.join(sentence)
         aux = stepwise_fit.get_params()
         par[sentence]=aux
         #forecasts = stepwise_fit.predict(test_data.shape[0])
         forecasts = stepwise_fit.predict_in_sample(test_data.shape[0])
         #mape_arima[sentence]=mean_absolute_percentage_error(list(list(test_data.VlTotalBruto)), list(forecasts))
         mape_arima[sentence]=MAPE(test_data.VlTotalBruto, forecasts)
         pred_arima[sentence]=forecasts
         smape_arima [sentence] = smape(test_data.VlTotalBruto, forecasts)
         

  
      # Ajustando modelo Croston
         croston_pred = croston.fit_croston(df+0.000001,test_data.shape[0],'original')

      # Armazenando resultados Croston
         #par_croston[sentence] = croston_pred
         #mape_croston[sentence] = mean_absolute_percentage_error(list(test_data.VlTotalBruto), croston_pred['croston_forecast'])
         #mape_croston[sentence] = MAPE(test_data.VlTotalBruto, croston_pred['croston_forecast'])
         #smape_croston[sentence] = smape(test_data.VlTotalBruto, pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0])
         #pred_croston[sentence] = pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0]
         
      # O modelo profet só funciona com duas colunas
         y = df.reset_index(drop=False)
         y.columns = ['ds', 'y']
         #y['floor'] = 0
         #y['cap'] = 200000
         train = y.iloc[:train_data.shape[0],:]
         test = y.iloc[train_data.shape[0]:,:]
         test=train

      # Ajustando modelo Prophet
         m = Prophet(weekly_seasonality = True,
                     yearly_seasonality = False,
                     daily_seasonality = False)
         m.fit(train,verbose=0)
         future = m.make_future_dataframe(periods=len(test))
         future['ds'] = pd.to_datetime(future['ds']).dt.date
         forecast = m.predict(future)
         one=test['y']
         twi=list(forecast.loc[:test.shape[0]-1,'yhat'])
         #mape_prophet[sentence] = mean_absolute_percentage_error(list(one),list(twi))# Assimetrico
         mape_prophet[sentence] = MAPE(one,twi)# Assimetrico
         smape_prophet[sentence] = smape(one, twi) # simmetrico
         pred_prophet[sentence] = twi
         


         # O EDM agora, preparando o imput
         y = y[["ds","y"]]
         y.columns = ["Time", "Revenue"]
         y.Time = list(range(1,df.shape[0]+1,1))
         
         #aaa=pyEDM.EmbedDimension(dataFrame = y, lib="1 53", pred="1 53", columns="Revenue",target = "Revenue",showPlot=False)
         #maxE = aaa.rho.max()
         #selectedE=aaa[aaa.rho==maxE]['E']
         #intervalo = pyEDM.PredictInterval(dataFrame = y,lib="1 53", pred="1 53", columns="Revenue", E=3, showPlot=False)
         #maxpred = intervalo.rho.max()
         #selectedpred=intervalo[intervalo.rho==maxpred]['Tp']
         simplex_pred = pyEDM.Simplex(dataFrame = y,lib="1 53", pred="1 53", E=3, columns="Revenue",Tp=0,tau=-1)
#print(pyEDM.ComputeError(simplex_pred['Observations'],simplex_pred['Predictions']))
         simplex_pred = simplex_pred[['Observations','Predictions']].dropna()
         pred_EDM[sentence] = list(simplex_pred['Predictions'])
         #mape_EDM[sentence] = mean_absolute_percentage_error(list(simplex_pred['Observations']), list(simplex_pred['Predictions']))
         mape_EDM[sentence] = MAPE(simplex_pred['Observations'], simplex_pred['Predictions'])
         smape_EDM[sentence] = smape(simplex_pred['Observations'], simplex_pred['Predictions'])
         
         # PAra Naive, exponential smothing, theta, FFT
         series = TimeSeries.from_dataframe(df.reset_index(), 'date', 'VlTotalBruto')

         #train2, val = series.split_after(1)
         train2 = val = series

         #trains2=val=series
         model1 = NaiveDrift()
         model1.fit(train2+0.00000001)
         pred_val1 = model1.predict(len(val))
         pred_Naive[sentence] = pd.DataFrame(pred_val1.values()).iloc[:,0]
         #mape_naive[sentence] = mean_absolute_percentage_error(val.values(),pred_val1.values())
         mape_naive[sentence] = MAPE(val.values(),pred_val1.values())
         smape_naive[sentence] = str(smape(val.values(),pred_val1.values()))
         
         #model2 = ExponentialSmoothing()
         #model2.fit(train2+0.00000001)
         #pred_val2 = model2.predict(len(val))
         #pred_ExponentialSmoothing[sentence] = pd.DataFrame(pred_val2.values()).iloc[:,0]
         #mape_ExponentialSmoothing[sentence] = mean_absolute_percentage_error(val.values(),pred_val2.values())
         #smape_ExponentialSmoothing[sentence] = str(smape(val.values(),pred_val2.values()))
         

         model3 = Theta()
         model3.fit(train2+0.00000001)
         pred_val3 = model3.predict(len(val))
         pred_Theta[sentence] = pd.DataFrame(pred_val3.values()).iloc[:,0]
         #mape_Theta[sentence] = mean_absolute_percentage_error(val.values(),pred_val3.values())
         mape_Theta[sentence] = MAPE(val.values(),pred_val3.values())
         smape_Theta[sentence] = str(smape(val.values(),pred_val3.values()))
         

         model4 = FFT(trend='poly',trend_poly_degree = 2)
         model4.fit(train2+0.00000001)
         pred_val4 = model4.predict(len(val))
         pred_FFT[sentence] = pd.DataFrame(pred_val4.values()).iloc[:,0]
         #mape_FFT[sentence] = mean_absolute_percentage_error(val.values(),pred_val4.values())
         mape_FFT[sentence] = MAPE(val.values(),pred_val4.values())
         smape_FFT[sentence] = str(smape(val.values(),pred_val4.values()))

    except:
     pass  

resultado_final_mape= pd.concat([pd.DataFrame.from_dict(mape_arima,orient='index'),
           #pd.DataFrame.from_dict(mape_croston,orient='index'),
           pd.DataFrame.from_dict(mape_prophet,orient='index'),
           pd.DataFrame.from_dict(mape_EDM,orient='index'),
           pd.DataFrame.from_dict(mape_naive,orient='index'),
           #pd.DataFrame.from_dict(mape_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(mape_Theta,orient='index'),
           pd.DataFrame.from_dict(mape_FFT,orient='index')   
           ], axis=1)
resultado_final_mape.columns=["arima", "prophet","EDM","naive","theta","fft"]

resultado_final_smape= pd.concat([pd.DataFrame.from_dict(smape_arima,orient='index'),
           #pd.DataFrame.from_dict(smape_croston,orient='index'),
           pd.DataFrame.from_dict(smape_prophet,orient='index'),
           pd.DataFrame.from_dict(smape_EDM,orient='index'),
           pd.DataFrame.from_dict(smape_naive,orient='index'),
           #pd.DataFrame.from_dict(smape_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(smape_Theta,orient='index'),
           pd.DataFrame.from_dict(smape_FFT,orient='index')   
           ], axis=1)
resultado_final_smape.columns=["arima", "prophet","EDM","naive","theta","fft"]

predicoes = pd.concat([pd.DataFrame.from_dict(pred_arima,orient='index'),
           #pd.DataFrame.from_dict(pred_croston,orient='index'),
           pd.DataFrame.from_dict(pred_prophet,orient='index'),
           pd.DataFrame.from_dict(pred_EDM,orient='index'),
           pd.DataFrame.from_dict(pred_Naive,orient='index'),
           #pd.DataFrame.from_dict(pred_ExponentialSmoothing,orient='index'),
           pd.DataFrame.from_dict(pred_Theta,orient='index'),
           pd.DataFrame.from_dict(pred_FFT,orient='index')   
           ], axis=0)
predicoes['modelos']= ["arima"]*len(pred_arima) + ["prophet"]*len(pred_prophet) + ["EDM"]*len(pred_EDM) + ['naive']*len(pred_Naive) + ['theta']*len(pred_Theta) + ['fft']*len(pred_FFT)



### Observados

In [ ]:
filter = df_2.columns.isin(list(cat_rest.index))
df3 = df_2.iloc[:,filter]
df3t = df3.T
df3t.columns = ['OBS_C_OUT' + str(i) for i in range (1, 54)]
df3tm = df3t.reset_index().melt(id_vars='index')
df3tm.columns = ['fornecedor','semana','valor']
df3tm['modelo'] = ['obs_com_out']*df3tm.shape[0]

In [ ]:
filter = df_2_sem_out.columns.isin(list(cat_rest.index))
df3_sem_out = df_2_sem_out.iloc[:,filter]
df3_sem_outt = df3_sem_out.T
df3_sem_outt.columns = ['OBS_S_OUT' + str(i) for i in range (1, 54)]
df3_sem_outtm = df3_sem_outt.reset_index().melt(id_vars='index')
df3_sem_outtm.columns = ['fornecedor','semana','valor']
df3_sem_outtm['modelo'] = ['obs_sem_out']*df3_sem_outtm.shape[0]

### Predicoes

In [ ]:
#df_empty = pd.DataFrame(index=predicoes.index, columns=range(0,50))
#predicoes_4 = pd.concat([df_empty,predicoes],axis=1)
#predicoes_4.columns = range(1,56)

In [ ]:
arima = predicoes[predicoes.modelos==predicoes.modelos.unique()[0]].iloc[:,:53].reset_index().melt(id_vars='index')
arima.columns = ['fornecedor','semana','valor']
arima['modelo'] = ['arima']*arima.shape[0]
arima['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,1]
arima['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,1]

propheta = predicoes[predicoes.modelos==predicoes.modelos.unique()[1]].iloc[:,:53].reset_index().melt(id_vars='index')
propheta.columns = ['fornecedor','semana','valor']
propheta['modelo'] = ['propheta']*propheta.shape[0]
propheta['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,2]
propheta['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,2]

EDM = predicoes[predicoes.modelos==predicoes.modelos.unique()[2]].iloc[:,:53].reset_index().melt(id_vars='index')
EDM.columns = ['fornecedor','semana','valor']
EDM['modelo'] = ['EDM']*EDM.shape[0]
EDM['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,3]
EDM['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,3]

naives = predicoes[predicoes.modelos==predicoes.modelos.unique()[3]].iloc[:,:53].reset_index().melt(id_vars='index')
naives.columns = ['fornecedor','semana','valor']
naives['modelo'] = ['naives']*naives.shape[0]
naives['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,4]
naives['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,4]

thetas = predicoes[predicoes.modelos==predicoes.modelos.unique()[4]].iloc[:,:53].reset_index().melt(id_vars='index')
thetas.columns = ['fornecedor','semana','valor']
thetas['modelo'] = ['thetas']*thetas.shape[0]
thetas['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,5]
thetas['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,5]

ffts = predicoes[predicoes.modelos==predicoes.modelos.unique()[5]].iloc[:,:53].reset_index().melt(id_vars='index')
ffts.columns = ['fornecedor','semana','valor']
ffts['modelo'] = ['ffts']*ffts.shape[0]
ffts['mape'] = pd.concat([resultado_final_mape.reset_index()]*53, ignore_index=True).iloc[:,6]
ffts['smape'] = pd.concat([resultado_final_smape.reset_index()]*53, ignore_index=True).iloc[:,6]

In [ ]:
RES_FINAL = pd.concat([df3tm, df3_sem_outtm, arima, propheta,EDM, naives, thetas, ffts],axis=0)

In [ ]:
RES_FINAL.to_excel('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/RES_FINAL.xlsx')

### Arrumando a matrix para agrupar os resultados

In [ ]:
predicoes.columns = predicoes.columns.droplevel()

In [ ]:
resul_pred = predicoes.reset_index().melt(id_vars=['index','modelos'])
resul_pred = resul_pred.reset_index().sort_values(['index','modelos','level_0'])


In [ ]:
resultado_final_mape.columns = resultado_final_mape.columns.droplevel()

In [ ]:
# 53 pois são 53 semanas
erro_mape = pd.concat([resultado_final_mape.reset_index().melt(id_vars='index')]*53, ignore_index=True)
erro_mape = erro_mape.sort_values(['index','variable'])

In [ ]:
resultado_final_smape.columns = resultado_final_smape.columns.droplevel()

In [ ]:
# 53 pois são 53 semanas
erro_smape = pd.concat([resultado_final_smape.reset_index().melt(id_vars='index')]*53, ignore_index=True)
erro_smape = erro_smape.sort_values(['index','variable'])

In [ ]:
df3t.columns = df3t.columns.droplevel()

In [ ]:
# 6 pois foram 6 ajustes de 6 modelos diferentes.
resul_obs = pd.concat([df3t.reset_index().melt(id_vars='index').sort_values(['index','SemanaAno'])]*6, ignore_index=True)

In [ ]:
RES_FINAL = pd.concat([resul_obs, resul_pred,erro_mape,erro_smape],axis=1)

In [ ]:
RES_FINAL.to_excel('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/RES_FINAL.xlsx')

In [ ]:
pd.concat([erro_mape,erro_smape.value],axis=1).to_excel('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/ERROS.xlsx')

### Agora para apenas Croston

In [ ]:
filter = df_original["DsDivisaoFornecedor"].isin(list(cat_croston.index))

df_original2 = df_original[filter]

dat_aux2 = pd.DataFrame(df_original2.groupby(['DsDivisaoFornecedor'])['CdGrupoProdutoSimilar'].unique())
dat_aux2

,CdGrupoProdutoSimilar
DsDivisaoFornecedor,
3M DO BRASIL LTDA - EPI,"[9000524.0, 9000527.0, 9000531.0]"
ADELBRAS IND COM ADESIVOS &&MP,"[408235.0, 408237.0, 408236.0]"
AGIS - LOGITECH,[2309279.0]
AGIS COMPONENTES INFORMATICA,"[2309636.0, 2309637.0]"
AGP TECNOLOGIA INFORM.BRASIL LTDA,[2309922.0]
...,...
VEGA STEEL IND LTDA MARTCON &&MP,"[1702981.0, 1702982.0, 1702979.0]"
VIAPOL LTDA,"[408446.0, 408457.0]"
VIQUA IND. DE PLASTICOS LTDA,"[4807515.0, 4807520.0, 4807505.0, 4807517.0, 4..."


In [ ]:
par_croston={}
mape_croston={}
smape_croston={}
maape_croston={}
pred_croston = {}






corte_serie=0.94
#for j, i in zip(tqdm (range(0, 7),desc="Processando Fornecedor", ascii=False, ncols=75), tqdm (range(0,len(dat_aux.iloc[j,0])),desc="Processando Id Oriduto", ascii=False, ncols=75)):

for j in tqdm (range(0, dat_aux2.shape[0]),desc="Processando Fornecedor", ascii=False, ncols=75):
  
    #try:
         
   # Selecionando os dados para ajustar os modelos
         
         filter1 = df_original2["DsDivisaoFornecedor"]==dat_aux2.index[j]
         #filter1 = df_original["DsDivisaoFornecedor"]=='NESTLE BRASIL - GAROTO'
         df_0 = df_original2[filter1]
         df = df_0.groupby(['SemanaAno'])[['VlTotalBruto']].sum()
         we = pd.DataFrame(list(range(0,num_de_semanas,1)))
         we.columns=['Semanas']
         we.index=we.Semanas
         df = pd.concat([df, we], axis=1)
         df=df['VlTotalBruto']
         df.index = pd.DataFrame(pd.date_range('2020-01-05', freq='7D', periods=num_de_semanas), columns=['date']).iloc[:,0]
         df = pd.DataFrame(df).fillna(0)
         

         if [mediana==0]:
           df.columns = ['VlTotalBruto']
    
    
         else:
           Q1 = df.quantile(0.25)
           Q3 = df.quantile(0.75)
           IQR = Q3 - Q1
           df[((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))] = mediana
           df.columns = ['VlTotalBruto']
     
   
          


      # Sets de treino e teste
         train_len = int(df.shape[0] * corte_serie)
         #train_len = int(corte_serie)
         train_data, test_data = df[:train_len], df[train_len:]
         #test_data = train_data
         test_data = train_data = df

      # Ajustando modelo Croston
         croston_pred = croston.fit_croston(df+0.000001,test_data.shape[0],'original')

      # Armazenando resultados Croston
         par_croston[dat_aux2.index[j]] = croston_pred
         mape_croston[dat_aux2.index[j]] = mean_absolute_percentage_error(list(test_data.VlTotalBruto), croston_pred['croston_forecast'])
         smape_croston[dat_aux2.index[j]] = smape(test_data.VlTotalBruto, pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0])
         pred_croston[dat_aux2.index[j]] = pd.DataFrame(croston_pred['croston_forecast']).iloc[:,0]
         

      
         
    #except:
     #pass  

Processando Fornecedor: 100%|████████████| 180/180 [00:11<00:00, 16.25it/s]


In [ ]:
pred_croston

In [ ]:
observado = df_2.loc[:,list(dat_aux2.index)].transpose()
observado.columns = range(0,53)
observado = observado.reset_index()
observado = pd.melt(observado, id_vars =['index'])
observado['modelo'] = ['observado']*observado.shape[0]

In [ ]:
predito = pd.DataFrame.from_dict(pred_croston).T.reset_index().melt(id_vars='index')
predito['modelo'] = ['predito']*predito.shape[0]

In [ ]:
observado.to_excel('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/obs_croston.xlsx')

In [ ]:
pd.concat([observado, predito],axis=0).to_excel('/content/drive/MyDrive/JOB-Martins/Previsão de Demanda/FINAL_CROSTON.xlsx')